# get_clk

**Author:** Marilyn Braojos Gutierrez\
**Purpose:** This program aims to download and uncompress RINEX format extension files with GPS satellite final clock bias information. These files contain the extension .clk.\
**PhD Milestone:** #1: *Leverage deep learning models to GPS satellite clock bias corrections.*\
**Project:** This program is Step (1) in this PhD milestone. Obtaining the data is the first critical step.\
**References:**\
[1] https://cddis.nasa.gov/Data_and_Derived_Products/GNSS/clock_products.html

# Import Libraries

In [5]:
from ftplib import FTP_TLS
import gzip
import matplotlib.pyplot as plt
import numpy as np
import os
from unlzw import unlzw

# Connecting to NASA's CDDIS FTP Server

In [7]:
# https://cddis.nasa.gov/Data_and_Derived_Products/GNSS/GNSS_data_and_product_archive.html

In [8]:
host = "gdc.cddis.eosdis.nasa.gov"                                    # FTP server host
port = 21                                                             # set port (21 is std port for FTP in passive mode)
username = "anonymous"
password = ""
timeout = 30                                                          # connection timeout (seconds)

In [9]:
ftp = FTP_TLS(timeout=timeout)
ftp.connect(host, port)
ftp.login(username, password)
ftp.prot_p()
ftp.retrlines('LIST')                                                 # list files in server

lrwxrwxrwx    1 ftp      ftp             8 Sep 06  2022 GSAC -> pub/GSAC
lrwxrwxrwx    1 ftp      ftp             9 Sep 06  2022 doris -> pub/doris
lrwxrwxrwx    1 ftp      ftp            11 Sep 06  2022 glonass -> pub/glonass
lrwxrwxrwx    1 ftp      ftp             7 Sep 06  2022 gnss -> pub/gps
lrwxrwxrwx    1 ftp      ftp             7 Sep 06  2022 gps -> pub/gps
drwxrwxr-x   26 ftp      ftp          4096 Jan 01  2024 highrate
lrwxrwxrwx    1 ftp      ftp             8 Sep 06  2022 misc -> pub/misc
lrwxrwxrwx    1 ftp      ftp            12 Sep 06  2022 products -> pub/products
drwxrwxr-x   13 ftp      ftp          4096 Feb 21  2023 pub
lrwxrwxrwx    1 ftp      ftp            11 Sep 06  2022 reports -> pub/reports
lrwxrwxrwx    1 ftp      ftp             7 Sep 06  2022 slr -> pub/slr
lrwxrwxrwx    1 ftp      ftp             8 Sep 06  2022 vlbi -> pub/vlbi


'226 Directory send OK.'

# Fetching and Uncompressing Final Clock Bias IGS GPS Data from FTP Server

In [11]:
def fetch_data(week):
    try:
        base_dir = '/pub/gps/products'
        ftp.cwd(base_dir)
            
        week_folder = f"{week}"
        ftp.cwd(week_folder)
        
        # List all files in the directory
            
        files = []
        ftp.retrlines('NLST', files.append)

        clk_files = [file for file in files if file.endswith('.clk.Z')] # CLK is case-sensitive
        # clk_files = [file for file in files if file.endswith('.CLK.gz')] # CLK is case-sensitive
            
        data_dir = 'clk'
        new_folder = os.path.join(data_dir, f'gps_{week}')
        os.makedirs(new_folder, exist_ok=True)
    
        for i, clk_file in enumerate(clk_files):
            local_filename = os.path.join(new_folder, clk_file)
                
            with open(local_filename, 'wb') as f:
                ftp.retrbinary(f"RETR {clk_file}", f.write)

            if clk_file.endswith('.gz'):
                local_uncompressed = local_filename[:-3]
                try: 
                    with gzip.open(local_filename, 'rb') as gzipped_file, open(local_uncompressed, 'wb') as decompressed_file:
                        decompressed_file.write(gzipped_file.read())
                        files.append(local_uncompressed)
                except Exception as e: 
                    print(f"Error uncompressing file {local_filename}: {e}. Will begin uncompressing file as a UNIX compressed file")
                    with open(local_filename, 'rb') as compressed_file, open(local_uncompressed, 'wb') as decompressed_file:
                        decompressed_file.write(unlzw(compressed_file.read()))
                        files.append(local_uncompressed)
                        print(f"Successfully uncompressed and saved file {local_filename} as a UNIX compressed file")

            
            if clk_file.endswith('.Z'):
                local_uncompressed = local_filename[:-2]
                try: 
                    with open(local_filename, 'rb') as compressed_file, open(local_uncompressed, 'wb') as decompressed_file:
                        decompressed_file.write(unlzw(compressed_file.read()))
                        files.append(local_uncompressed)
                except Exception as e: 
                    print(f"Error uncompressing file {local_filename}: {e}. Will begin uncompressing file as a UNIX compressed file")
                            
            print(f"Done saving file {i + 1} out of {len(clk_files)} for week: {week}")
        print(f"Done uncompressing and saving files for week: {week}")      

    except Exception as e:
        print(f"Error fetching data for week {week}: {e}")

In [12]:
%%time 

# https://beta.ngs.noaa.gov/CORS/Gpscal.shtml
weeks = range(2069,2087)

for week in weeks: 
    fetch_data(week)

Done saving file 1 out of 70 for week: 2069
Done saving file 2 out of 70 for week: 2069
Done saving file 3 out of 70 for week: 2069
Done saving file 4 out of 70 for week: 2069
Done saving file 5 out of 70 for week: 2069
Done saving file 6 out of 70 for week: 2069
Done saving file 7 out of 70 for week: 2069
Done saving file 8 out of 70 for week: 2069
Done saving file 9 out of 70 for week: 2069
Done saving file 10 out of 70 for week: 2069
Done saving file 11 out of 70 for week: 2069
Done saving file 12 out of 70 for week: 2069
Done saving file 13 out of 70 for week: 2069
Done saving file 14 out of 70 for week: 2069
Done saving file 15 out of 70 for week: 2069
Done saving file 16 out of 70 for week: 2069
Done saving file 17 out of 70 for week: 2069
Done saving file 18 out of 70 for week: 2069
Done saving file 19 out of 70 for week: 2069
Done saving file 20 out of 70 for week: 2069
Done saving file 21 out of 70 for week: 2069
Done saving file 22 out of 70 for week: 2069
Done saving file 23

# Archive

In [14]:
# def fetch_data(week):
#         try:
#             base_dir = '/pub/gps/products'
#             ftp.cwd(base_dir)
            
#             week_folder = f"{week}"
#             ftp.cwd(week_folder)
        
#             # List all files in the directory
            
#             files = []
#             ftp.retrlines('NLST', files.append)
    
#             clk_files = [file for file in files if file.endswith('.CLK.gz')] # CLK is case-sensitive
            
#             data_dir = 'clk'
#             new_folder = os.path.join(data_dir, f'gps_{week}')
#             os.makedirs(new_folder, exist_ok=True)
    
#             for i, clk_file in enumerate(clk_files):
#                 local_filename = os.path.join(new_folder, clk_file)
                
#                 with open(local_filename, 'wb') as f:
#                     ftp.retrbinary(f"RETR {clk_file}", f.write)

#                 if clk_file.endswith('.gz'):
#                     local_uncompressed = local_filename[:-3]
#                     try: 
#                         with gzip.open(local_filename, 'rb') as gzipped_file, open(local_uncompressed, 'wb') as decompressed_file:
#                             decompressed_file.write(gzipped_file.read())
#                             files.append(local_uncompressed)
#                     except Exception as e: 
#                         print(f"Error uncompressing file {local_filename}: {e}. Will begin uncompressing file as a UNIX compressed file")
#                         with open(local_filename, 'rb') as compressed_file, open(local_uncompressed, 'wb') as decompressed_file:
#                             decompressed_file.write(unlzw(compressed_file.read()))
#                             files.append(local_uncompressed)
#                             print(f"Successfully uncompressed and saved file {local_filename} as a UNIX compressed file")
                            
#                 print(f"Done saving file {i + 1} out of {len(clk_files)} for week: {week}")
#             print(f"Done uncompressing and saving files for week: {week}")      

#         except Exception as e:
#             print(f"Error fetching data for week {week}: {e}")